In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

Data Loading

In [2]:

def load_data(file):
    mdro = pd.read_excel(file, sheet_name='MDROs', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    report = pd.read_excel(file, sheet_name='微生物報告', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    info = pd.read_excel(file, sheet_name='就醫資訊', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    anti = pd.read_excel(file, sheet_name='抗生素使用', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    examValue = pd.read_excel(file, sheet_name='檢驗數值', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    
    # data processing 
    mdro = mdro[['病歷號碼', 'time', '就醫序號','申請編號', '菌株', '預測值', '部位', 'sendTime']]
    mdro.rename({'time':'openTime'}, axis=1, inplace=True)
    
    report = report[['病歷號碼', 'bed', '就醫序號','申請編號', 'strain','部位', '抗藥性菌株', 'examTime','reportTime']]
    
    cdss = pd.merge(report, mdro, how='left', on=['病歷號碼', '就醫序號','申請編號', '部位'])
    cdss = cdss.drop_duplicates()
    cdss['minTime'] = cdss.groupby(['病歷號碼', 'bed','就醫序號', '申請編號', '部位', '菌株', '預測值',  'sendTime'])['openTime'].transform('min')
    
    info = info[['病歷號碼', '就醫序號', '性別', '生日', '年齡', '來院日期', '離院日期', '天數', '門急住', '科別代碼', '科別']]
    info.rename({'門急住':'bed'}, axis=1, inplace=True)
    
    cdss_info = pd.merge(cdss, info, how='left', on=['病歷號碼', '就醫序號', 'bed'])
    cdss_info = cdss_info.drop_duplicates()
    
    CDSS = pd.merge(cdss_info, anti, how='left', on=['病歷號碼','就醫序號'])
    CDSS = CDSS.drop_duplicates()
    
    
    return CDSS
    

In [4]:
# 智抗菌平台報表_20220331.xlsx

file = r'C:\Users\User\VSCode\智抗菌平台報表_20220331.xlsx'
CDSS = load_data(file)
# CDSS.head(10)

In [ ]:
CDSS_old = pd.read_excel(r'C:\Users\User\VSCode\CDSS(2022).xlsx')
CDSS = pd.concat([CDSS_old, CDSS])
CDSS = CDSS.drop_duplicates()